In [1]:
import pyro
import torch
pyro.set_rng_seed(101)

import pyro.distributions as dist
from pyro.infer import Importance, EmpiricalMarginal
import matplotlib.pyplot as plt

import numpy as np
from collections import Counter
from googlesearch import search
from ipywidgets import interactive
from IPython.display import display

In [2]:
alias = {'entity1':['person','animal'],
         'entity2':['person','animal'],
         'position1':['left','right'], 
         'position2':['left','right'], 
         'haircolor':['black', 'brown','red','golden'],
         'hairlength':['short','long'],
         'gender':['male','female'],
         'age':['young','old'],
         'color':['black', 'white', 'brown'],
         'size':['little','large'],
         'type':['cat','dog'],
         'background':['park','home'],
         'interaction':['fight','chase','call','hold']
        }

prob = {'entity1':torch.tensor([0.5,0.5]), 
        'entity2':torch.tensor([0.5,0.5]), 
        'position1':torch.tensor([0.5,0.5]), 
        'position2':torch.tensor([0.5,0.5]), 
        'haircolor':torch.tensor([0.25, 0.25,0.25,0.25]),
        'hairlength':torch.tensor([0.5,0.5]),
        'gender':torch.tensor([0.5,0.5]),
        'age':torch.tensor([0.5,0.5]),
        'color':torch.tensor([1/3,1/3,1/3]),
        'size':torch.tensor([0.5,0.5]),
        'type':torch.tensor([0.5,0.5]),
        'background':torch.tensor([0.5,0.5]),
        'interaction':torch.tensor([[[1/2,0,0,1/2],[0,1/2,1/2,0]], # e1=left, [e2=left. e2=right]
                                     [[0,1/2,1/2,0],[1/2,0,0,1/2]]]) # e1=right, [e2=left. e2=right]
       }

causal_var = ['position1', 'position2', 'background', 'interaction']
other_var = ['haircolor', 'hairlength', 'gender', 'age', 'color', 'size', 'type']

In [3]:
class Person(object):
    def __init__(self):
        self.position1 = pyro.sample("position1", dist.Categorical(probs=prob['position1'])) 
        self.haircolor = pyro.sample("haircolor", dist.Categorical(probs=prob['haircolor']))
        self.hairlength = pyro.sample("hairlength", dist.Categorical(probs=prob['hairlength']))
        self.gender = pyro.sample("gender", dist.Categorical(probs=prob['gender']))
        self.age = pyro.sample("age", dist.Categorical(probs=prob['age']))

In [4]:
class Animal(object):
    def __init__(self):
        self.position2 = pyro.sample('position2', dist.Categorical(prob['position2']))
        self.color = pyro.sample('color', dist.Categorical(prob['color']))
        self.size = pyro.sample('size', dist.Categorical(prob['size']))
        self.type = pyro.sample('type', dist.Categorical(prob['type']))

In [5]:
def generateEntity():
    entity = pyro.sample("entity", dist.Categorical(probs=prob['entity']))
    if entity == 0:
        return Person()
    else:
        return Animal()

In [6]:
class Background(object):
    def __init__(self):
        self.background = pyro.sample('background', dist.Categorical(prob['background']))

In [7]:
class Interaction(object):
    def __init__(self, entity1, entity2):
        self.interaction = pyro.sample('interaction', dist.Categorical(prob['interaction'][entity1.position1][entity2.position2]))

In [8]:
def model():
    person = Person()
    animal = Animal()
    background = Background()
    interaction = Interaction(person, animal)

In [9]:
def condition(model, evidence, num_samples = 1000):
    condition_model = pyro.condition(model, data=evidence)
    posterior = pyro.infer.Importance(condition_model, num_samples=num_samples).run()
    return posterior

In [10]:
def intervention(model, evidence, num_samples = 1000):
    do_model = pyro.do(model, data=evidence)
    posterior = pyro.infer.Importance(do_model, num_samples=num_samples).run()
    return posterior

In [11]:
def pltDistribution(posterior, infer, num_samples = 1000):
    for i in range(len(infer)):
        marginal = EmpiricalMarginal(posterior, infer[i])
        samples = [marginal().item() for _ in range(num_samples)]
        unique, counts = np.unique(samples, return_counts=True)
        plt.figure(i)
        plt.bar(unique, counts / num_samples, align='center')
        plt.xticks(unique, alias[infer[i]])
        plt.ylabel('prob')
        plt.xlabel(infer[i])

In [12]:
def mostOccurance(posterior, infer, num_samples = 1000):
    sample = []
    for i in range(num_samples): 
        trace = posterior()
        value = []
        for i in range(len(infer)):
            value.append(trace.nodes[infer[i]]['value'])
        sample.append(tuple(value))

    most_common, num_most_common = Counter(sample).most_common(1)[0]
    
    infer_dict = {}
    for i in range(len(infer)):
        infer_dict[infer[i]] = alias[infer[i]][most_common[i]]
    return infer_dict

In [13]:
def getEntity(input_dic):
    dic = {}
    person = Person()
    animal = Animal()
    background = Background()
    interaction = Interaction(person,animal)
    entity_list = [person,animal,background,interaction]
    for k in entity_list:
        entity_dic = k.__dict__
        dic.update(entity_dic)
        
    for k in dic:
        dic[k] = alias[k][dic[k]]
    dic.update(input_dic)
    return dic

In [14]:
def getScene(input_dict):
    scene = str("A %s %s with %s %s hair %s a %s %s %s %s." % 
                (input_dict['age'],input_dict['gender'],input_dict['haircolor'],input_dict['hairlength'],
                 input_dict['interaction'],input_dict['size'],input_dict['color'],input_dict['type'],
                 input_dict['background']))
    print(scene)
    return scene

In [15]:
def getPic(input_str,num_s=3):
    for j in search(input_str, tld="com", num = num_s, start = 0, stop=5, pause=2): 
        print(j) 

In [16]:
def f_interactive(position1,position2,haircolor,hairlength,gender,age,color,size,type,background,interaction):
    pass

w = interactive(f_interactive, 
                   position1=sum([["-"],alias['position1']], []),
                   position2=sum([["-"],alias['position2']], []),
                   haircolor=sum([["-"],alias['haircolor']], []),
                   hairlength=sum([["-"],alias['hairlength']], []),
                   gender=sum([["-"],alias['gender']], []),
                   age=sum([["-"],alias['age']], []),
                   color=sum([["-"],alias['color']], []),
                   size=sum([["-"],alias['size']], []),
                   type=sum([["-"],alias['type']], []),
                   background=sum([["-"],alias['background']], []),
                   interaction=sum([["-"],alias['interaction']], []))

In [17]:
def getVar(w):
    do_var = w.kwargs
    do_causal_var = {}
    do_other_var = {}
    for var in causal_var:
        if do_var[var] == '-':
            continue
        else:
            do_causal_var[var] = torch.tensor(alias[var].index(do_var[var]))
    for var in other_var:
        if do_var[var] == '-':
            continue
        else:
            do_other_var[var] = torch.tensor(alias[var].index(do_var[var]))
    return do_causal_var, do_other_var

In [18]:
def getInfer(do_causal_var):
    do_causal_var_key = list(do_causal_var.keys())
    for var in do_causal_var_key:
        if var in infer_all:
            infer_all.remove(var)

In [19]:
display(w)

interactive(children=(Dropdown(description='position1', options=('-', 'left', 'right'), value='-'), Dropdown(d…

In [20]:
do_causal_var, do_other_var = getVar(w)
infer_all = ['interaction', 'background']
infer = getInfer(do_causal_var)
infer_model = condition(model, evidence = do_causal_var)
infer_res = {}
if infer:
    pltDistribution(infer_model, infer)
    infer_res = mostOccurance(infer_model, infer)
getPic(getScene(getEntity(infer_res)))

A young male with black long hair chase a little brown dog home.
http://trianglelostpets.org/searchs.php
http://trianglelostpets.org/searchr.php
